In [2]:
import numpy as np
import time
import scipy 
from scipy.linalg import khatri_rao
import itertools
from tqdm.notebook import tqdm

In [196]:
def tensor_from_tucker(core, factors):
    ndim = core.ndim
    res = core.copy()
    for k in range(ndim):
        res = np.tensordot(res, factors[k], axes=([k], [0]))
        res = np.moveaxis(res, -1, k)
    return res

In [243]:
def hosvd(tensor, rank, verbose=False):
    ndim = tensor.ndim
    dims = tensor.shape
    factors = []
    if isinstance(rank, int):
        rank = [rank] * ndim
    if verbose:
        print('ranks', rank)
    
    for k in range(ndim):
        A_k = np.moveaxis(tensor, k, 0).reshape(dims[k], -1)
        U, _, _ = scipy.sparse.linalg.svds(A_k, k=rank[k])
        factors.append(U.T)
        
    core = tensor.copy()
    for k in range(ndim):
        if verbose:
            print(k, core.shape, factors[k].shape)
        core = np.tensordot(core, factors[k].T, axes=([k], [0]))
        core = np.moveaxis(core, -1, k)
    
    if verbose:
        print("Факторы:")
        for f in factors:
            print(f.shape)
        print("Ядро:", core.shape)
        
    return core, factors

In [295]:
def st_hosvd(tensor, rank, verbose=False):
    ndim = tensor.ndim
    dims = list(tensor.shape)
    factors = []
    if isinstance(rank, int):
        rank = [rank] * ndim
    if verbose:
        print('ranks', rank)
    
    core = tensor.copy()
    for k in range(ndim):
        X_k = np.moveaxis(core, k, 0).reshape(dims[k], -1)
        U, s, V = scipy.sparse.linalg.svds(X_k, k=rank[k])
        factors.append(U.T)
        core = np.diag(s) @ V
        
        new_dims = [rank[k]] + dims[:k] + dims[k+1:]
        core = core.reshape(new_dims)
        dims[k] = rank[k]
        core = np.moveaxis(core, 0, k)
    
    if verbose:
        print("Факторы:")
        for f in factors:
            print(f.shape)
        print("Ядро:", core.shape)
    return core, factors

In [313]:
def st_hosvd_err(tensor, err, verbose=False):
    ndim = tensor.ndim
    dims = list(tensor.shape)
    factors = []
    ranks = []
    core = tensor.copy()
    errors = []
    cur_err = err / np.sqrt(ndim)
    for k in range(ndim):
        cur_err = err*err
        for e in errors:
            cur_err -= e*e
        cur_err = np.sqrt(cur_err/(ndim - k))
        errors.append(cur_err)
        
        X_k = np.moveaxis(core, k, 0).reshape(dims[k], -1)
        U, s, V = np.linalg.svd(X_k, full_matrices=False)
        sing_sum = 0.0
        i = s.shape[0]
        while i > 0 and sing_sum + s[i-1]*s[i-1] < cur_err:
            i -= 1
            sing_sum += s[i]*s[i]
        if i < 1:
            print('something is very bad')
        r = i
        U = U[:,:r]
        s = s[:r]
        V = V[:r,:]
        
        factors.append(U.T)
        core = np.diag(s) @ V
        ranks.append(r)
        new_dims = [r] + dims[:k] + dims[k+1:]
        core = core.reshape(new_dims)
        dims[k] = r
        core = np.moveaxis(core, 0, k)
    
    
    if verbose:
        print('Ранги:', ranks)
        print('Факторы:')
        for f in factors:
            print(f.shape)
        print('Ядро:', core.shape)
    return core, factors

In [314]:
sizes = np.array((20, 30, 40))
tensor = np.zeros(sizes)
for i in range(sizes[0]):
    for j in range(sizes[1]):
        for k in range(sizes[2]):
            tensor[i, j, k] = np.sin(i + j + k+20)
rank = [2, 3, 4]
st = time.time()
core, factors = hosvd(tensor, rank, verbose=1)
tucker_tensor = tensor_from_tucker(core, factors)
print(f'hosvd time = {time.time() - st:.5f} seconds')
err = np.linalg.norm(tensor - tucker_tensor)
print(f'error = {err}')


rank = [2, 2, 2]
st = time.time()
core, factors = hosvd(tensor, rank, verbose=1)
tucker_tensor = tensor_from_tucker(core, factors)
print(f'hosvd time = {time.time() - st:.5f} seconds')
err = np.linalg.norm(tensor - tucker_tensor)
print(f'error = {err}')

rank = [2, 3, 4]
st = time.time()
core, factors = st_hosvd(tensor, rank, verbose=1)
tucker_tensor = tensor_from_tucker(core, factors)
print(f'st-hosvd time = {time.time() - st:.5f} seconds')
err = np.linalg.norm(tensor - tucker_tensor)
print(f'error = {err}')

rank = [2, 2, 2]
st = time.time()
core, factors = st_hosvd(tensor, rank, verbose=1)
tucker_tensor = tensor_from_tucker(core, factors)
print(f'st-hosvd time = {time.time() - st:.5f} seconds')
err = np.linalg.norm(tensor - tucker_tensor)
print(f'error = {err}')

st = time.time()
core, factors = st_hosvd_err(tensor, err)
tucker_tensor = tensor_from_tucker(core, factors)
print(f'st-hosvd_err time = {time.time() - st:.5f} seconds')
err = np.linalg.norm(tensor - tucker_tensor)
print(f'error = {err}')

ranks [2, 3, 4]
0 (20, 30, 40) (2, 20)
1 (2, 30, 40) (3, 30)
2 (2, 3, 40) (4, 40)
Факторы:
(2, 20)
(3, 30)
(4, 40)
Ядро: (2, 3, 4)
hosvd time = 0.00571 seconds
error = 1.7828753384011e-13
ranks [2, 2, 2]
0 (20, 30, 40) (2, 20)
1 (2, 30, 40) (2, 30)
2 (2, 2, 40) (2, 40)
Факторы:
(2, 20)
(2, 30)
(2, 40)
Ядро: (2, 2, 2)
hosvd time = 0.00275 seconds
error = 1.6357524374015027e-13
ranks [2, 3, 4]
Факторы:
(2, 20)
(3, 30)
(4, 40)
Ядро: (2, 3, 4)
st-hosvd time = 0.00161 seconds
error = 1.261595353289613e-13
ranks [2, 2, 2]
Факторы:
(2, 20)
(2, 30)
(2, 40)
Ядро: (2, 2, 2)
st-hosvd time = 0.00732 seconds
error = 7.079951298614717e-14
st-hosvd_err time = 0.00246 seconds
error = 2.7123315182126884e-13


In [320]:
sizes = np.array((8,8,8,8,8,8,8,8))
tensor = np.zeros(sizes)
for I in itertools.product(*(range(i) for i in sizes)):
    tensor[I] = np.sin(sum(I))
    
rank = 2
st = time.time()
core, factors = hosvd(tensor, rank)
tucker_tensor = tensor_from_tucker(core, factors)
print(f'hosvd time = {time.time() - st:.5f} seconds')
err = np.linalg.norm(tensor - tucker_tensor)
print(f'error = {err}')

rank = 2
st = time.time()
core, factors = st_hosvd(tensor, rank)
tucker_tensor = tensor_from_tucker(core, factors)
print(f'st-hosvd time = {time.time() - st:.5f} seconds')
err = np.linalg.norm(tensor - tucker_tensor)
print(f'error = {err}')

st = time.time()
core, factors = st_hosvd_err(tensor, err/1000)
tucker_tensor = tensor_from_tucker(core, factors)
print(f'st-hosvd_err time = {time.time() - st:.5f} seconds')
err = np.linalg.norm(tensor - tucker_tensor)
print(f'error = {err}')

hosvd time = 1.84646 seconds
error = 1.5272250292813612e-08
st-hosvd time = 0.29831 seconds
error = 2.3928354921929565e-09
st-hosvd_err time = 1.15792 seconds
error = 3.373054578415253e-08
